Comparar KMeans com Improved Density Canopy com KMeans "puro" e outras variações.

Métricas erro Quadrático médio

1. Calcular MeanDis de cada elemento
2. Calcular densidade de cada elemento
3. O com maior densidade é o centro e eliminar os que não estão dentro do raio.

In [1]:
import pandas as pd
import numpy as np
from time import time

from IPython.display import display

from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import pairwise_distances

In [2]:
# X shoudl be a numpy matrix, very likely sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix
# T1 > T2 for overlapping clusters
# T1 = Distance to centroid point to not include in other clusters
# T2 = Distance to centroid point to include in cluster
# T1 > T2 for overlapping clusters
# T1 < T2 will have points which reside in no clusters
# T1 == T2 will cause all points to reside in mutually exclusive clusters
# Distance metric can be any from here: http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.pairwise_distances.html
# filemap may be a list of point names in their order in X. If included, row numbers from X will be replaced with names from filemap. 
 
def canopy(X, T1, T2, distance_metric='euclidean', filemap=None):
    canopies = dict()
    X1_dist = pairwise_distances(X, metric=distance_metric)
    canopy_points = set(range(X.shape[0]))
    while canopy_points:
        point = canopy_points.pop()
        i = len(canopies)
        canopies[i] = {"c":point, "points": list(np.where(X1_dist[point] < T2)[0])}
        canopy_points = canopy_points.difference(set(np.where(X1_dist[point] < T1)[0]))
    if filemap:
        for canopy_id in canopies.keys():
            canopy = canopies.pop(canopy_id)
            canopy2 = {"c":filemap[canopy['c']], "points":list()}
            for point in canopy['points']:
                canopy2["points"].append(filemap[point])
            canopies[canopy_id] = canopy2
    return canopies

In [15]:
def euclideanDistance(vector1, vector2):
        #print(vector1)
        #print(vector2)
        return np.sqrt(np.sum(np.power(vector1-vector2, 2)))

def getDistance(row_center, row_sample):
        #print(row_center)
        row_center = np.asarray(row_center)
        #row_center = np.asarray(row_sample)
        return euclideanDistance(row_center, row_sample)

def getSquaredError(data, kmeans):
    distances = []
    for i in range(k): # Qtd de clusters
        distance = 0
        for index_labels, value_labels in enumerate(kmeans.labels_): #kmeans.labels_ possui a que cluster cada elemento pertence
            if (i == value_labels):
                #print(value_labels)
                distance = distance + getDistance(kmeans.cluster_centers_[value_labels], data.loc[index_labels].values)
        
        distances.append(distance) #Erro quadratico medio de cada cluster
    
    distances = np.asarray(distances)
    error = np.sum(distances)
    return error

In [88]:
# data = pd.read_csv("datasets/iris.data", header=None)
# target = data.iloc[:,-1]
# data = data.iloc[:,:-1]
# meanDis = mean_dist(data)
# den = get_densities(data, meanDis)
# As = cluser_dist_mean(data, den, meanDis)
# clusters_dist(data, den)

In [89]:
# -------------- Density Canopy -------------- #

# Definition 1
#OBS.: enumerate com numpy mto mais rápido que iterrows
def mean_dist(D):
    n = D.shape[0]
    D = D.values
    sum_D = np.zeros((n, n))
    for i, row_i in enumerate(D):
        for j, row_j in enumerate(D[i+1:,]):
            sum_D[i][j] = euclideanDistance(row_i, row_j)
    return (2/(n*(n-1))) * np.sum(sum_D)

# Definition 2
def get_densities(D, meanDis):
    densities = np.zeros(D.shape[0], dtype=int)
    aux_D = D.values
    for i, row_i in enumerate(aux_D):
        for j, row_j in enumerate(aux_D):
            if euclideanDistance(row_i, row_j) - meanDis < 0:
                densities[i] += 1
    df = pd.DataFrame(data=densities, columns=["density"])
    return df

#Definition 3
def cluser_dist_mean(D, densities, meanDis):
    a = np.zeros(D.shape[0])
    densities_aux = densities.copy().values
    for i, row_i in enumerate(D.values):
        sum_dists = 0
        for j, row_j in enumerate(D.values):
            dist = euclideanDistance(row_i, row_j)
            if dist - meanDis < 0:
                sum_dists += dist
        a[i] = (2/(densities_aux[i]*(densities_aux[i]-1))) * sum_dists
    return a

#Definition 4
def clusters_dist(D, densities):
    s = []
    densities_aux = densities.values
    for i, row_i in enumerate(D.values):
        maxDist = 0
        minDist = float("inf")
        dist = 0
        flag = 1 #Se flag=0 entao min dist, se flag=1 retornar max dist
        for j, row_j in enumerate(D.values):
            dist = euclideanDistance(row_i, row_j)
            if densities_aux[j] > densities_aux[i]:
                flag = 0
                if (dist < minDist):
                    minDist = dist
            else:
                if (dist > maxDist):
                    maxDist = dist
#             if densities[j] == np.amax(densities):
#                 flag = 1
        if flag == 1:
            s.append(maxDist)
        else: # p(j) > p(i)
            s.append(minDist)
    return s

# Definition 6
def product_weight(p, a, s):
    w = []
    for i, row_i in enumerate(p):
        w.append(p[i] * (1/a[i]) * s[i])
    return w
        
def getCluster(D, meanDis, firstExecution=True, index=None): 
    aux_D = D.copy()
    df = get_densities(D, meanDis)
    if (firstExecution == True): #Primeira execução pega o de maior densidade
        rows = D.iloc[df.idxmax(axis=0)].values
        row_i = rows[-1] #pegando o último index com maior peso, caso tenha mais de 1
    else:
        row_i = D.iloc[index].values #pegando index passado, no caso o com maior peso
    cluster = []
    for j, row_j in enumerate(aux_D.values):
        if euclideanDistance(row_i, row_j) - meanDis < 0:
            cluster.append(j)
    #print (cluster)
    #auxD.drop(cluster, inplace=True)
    #display(auxD)
    return row_i, cluster #Elemento central e cluster

def removeOutliers(aux_D, densities, s, meanDis):
    #remove elemento com densidade = 1 e que o s[i] seja maior que o raio
    outliers = []
    for i, row_i in enumerate(aux_D.values):
        if densities[i] == 1 and s[i] > meanDis:
            outliers.append(i)
    aux_D.drop(outliers, inplace=True) #removendo outliers
    aux_D.reset_index(drop=True, inplace=True)
    densities = np.delete(densities, outliers, 0)
    s = np.delete(s, outliers, 0)
    return aux_D, densities, s

def run(D):
    meanDis = mean_dist(D)
    centers, cluster = getCenter(D, meanDis)
    centers = np.array([centers])
    aux_D = D.copy()
    aux_D.drop(cluster, inplace=True) #removendo cluster ja identificado
    aux_D.reset_index(drop=True, inplace=True)
    while (not auxD.empty):
        new_densities = get_densities(aux_D, meanDis)
        a = cluser_dist_mean(aux_D, new_densities, meanDis) #mantem-se meanDis
        s = clusters_dist(aux_D, new_densities)
        densities = new_densities.values
        aux_D, densities, s = removeOutliers(aux_D, densities, s, meanDis)
        aux_index = np.argmax(product_weight(densities, a, s))
#         for i, row_i in enumerate(aux_D.values):
#             if (densities[i] * (1/a[i]) * s[i]) > weight:
#                 weight = densities[i] * (1/a[i]) * s[i]
#                 print("densidade: ", densities[i], "a: ", a[i], "s: ", s[i], "weight: ", weight)
#                 aux_index = i
        aux_center, cluster = getCenter(aux_D, meanDis, firstExecution=False, index=aux_index)
        aux_center = np.array([aux_center])
        centers = np.concatenate((centers, aux_center), axis=0)
        aux_D.drop(cluster, inplace=True) #removendo cluster ja identificado
        aux_D.reset_index(drop=True, inplace=True)
    print("KS:", len(centers), centers)

### possiveis problemas que podem estar ocorrendo 
1. em getAs a função estar errada
2. Checar se os valores dropados estão corretos
3. Ver se foi retirado o index tb
4. Checar se esses limites de outliers são suficientes
5. Checar se algum índice não está estranho

In [5]:
files = ["soybean-small", "iris", "wine",  "segmentation", "ionosphere"]
ks = [4,3,3,7,2]
#kmeansTypes = ["random", "k-means++"]
kmeansTypes = ["random"]

In [6]:
for kmeansType in kmeansTypes:
    print ("--------- "+ kmeansType +" test ---------")
    
    for index, file in enumerate(files):
        print ("\n----- "+file+" -----\n")
        data = pd.read_csv("datasets/"+file+".data", header=None)
        print (data.shape)

        #display(data.head())
        if file == "segmentation": #Target eh na primeira coluna
            target = data.iloc[:,0]
            data = data.iloc[:,1:]
            
            
        else: #Target na última coluna
            target = data.iloc[:,-1]
            data = data.iloc[:,:-1]
        
        
        getWeight(data)
        
        clustering_times = []
        start = time()
        k = ks[index]
        kmeans = KMeans(n_clusters=k, random_state=100, init=kmeansType, n_init=1, max_iter=100).fit(data)
        #display(data)
        error = getSquaredError(data, kmeans)
        
        
                #print(kmeans.labels_)
        end = time()
        T1 = error/(data.shape[0])
        T2 = error/(data.shape[0]/2)
        
        #print(canopy(data.values, T1, T2))
        #print("Erro quadrático médio: ",error)
        clustering_times.append(end - start)
        #print(clustering_times)
        #print(kmeans.cluster_centers_)

--------- random test ---------

----- soybean-small -----

(47, 36)
densidade:  [4] a:  [1.42544879] s:  6.928203230275509 weight:  [19.44146505]
densidade:  [4] a:  [1.25233081] s:  7.0 weight:  [22.35830967]
densidade:  [10] a:  [0.68063463] s:  6.855654600401044 weight:  [100.72444602]
densidade:  [10] a:  [0.51850195] s:  6.324555320336759 weight:  [121.97746411]
densidade:  [4] a:  [1.42544879] s:  3.3166247903554 weight:  [9.30689283]
KS: 3

----- iris -----

(150, 5)
densidade:  [24] a:  [0.05370115] s:  6.498461356351979 weight:  [2904.27787241]
densidade:  [24] a:  [0.04989528] s:  6.717886572427373 weight:  [3231.35335588]
densidade:  [16] a:  [0.1564202] s:  1.8165902124584954 weight:  [185.81643419]
densidade:  [16] a:  [0.09518695] s:  1.284523257866513 weight:  [215.91585604]
densidade:  [16] a:  [0.10844554] s:  1.5165750888103098 weight:  [223.75471445]
densidade:  [16] a:  [0.11142851] s:  1.562049935181331 weight:  [224.29448082]
KS: 3

----- wine -----

(178, 14)
de

In [7]:
#usando numpy em vez do pandas pode tornar mais rápido pra essas operações